In [4]:
import pandas as pd
import os
from os.path import join    
sp_dir = '/Users/atecza/Library/CloudStorage/OneDrive-RMI/Documents/OCI Phase 2/Midstream/PRELIM_v1.6.xlsm'

In [65]:
def extract_assay_info(file):
    
    df = pd.read_excel(file,sheet_name='Assay Inventory', na_values=['NA'], usecols="B:E",skiprows=7, names=['Country', 'Selected Assay', 'Units', 'Full Crude'])
    
    country_list = []
    assay_names = []
    assay_numbers = []
    units_flow = []
    units_sulphur = []
    units_api = []
    full_crude_vol_flows = []
    full_crude_sulphurs = []
    full_crude_apis = []

    for i in df.index:

        if pd.notnull(df.at[i,'Country']):

            country_list.append(df.at[i,'Country'])
            assay_names.append(df.at[i,'Selected Assay'])
            assay_numbers.append(df.at[i+1,'Units'])

            #get vol flow
            units_flow.append(df.at[i+3,'Units'])
            full_crude_vol_flows.append(df.at[i+3,'Full Crude'])

            #get sulpher
            units_sulphur.append(df.at[i+6,'Units'])
            full_crude_sulphurs.append(df.at[i+6,'Full Crude'])

            #get api
            units_api.append(df.at[i+8,'Units'])
            full_crude_apis.append(df.at[i+8,'Full Crude'])



    df_assay = pd.DataFrame(list(zip(country_list, assay_names, assay_numbers, units_flow, full_crude_vol_flows, units_sulphur, full_crude_sulphurs,units_api,full_crude_apis)), columns=['Country','Assay Name','Assay Number','Throughput Unit','Throughput','Sulphur Unit','Sulphur','API Unit','API'])
    
    return df_assay
        